In [2]:
import cv2
import os
import numpy as np

In [3]:
mask_image = cv2.imread('mickey.png')

if mask_image is None:
    print(f"Error: Unable to load mask image from {mask_image}. Check the file path and try again.")
    exit()

In [4]:
cascPathface = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
cascPatheyes = os.path.dirname(cv2.__file__) + "/data/haarcascade_eye_tree_eyeglasses.xml"

faceCascade = cv2.CascadeClassifier(cascPathface)
eyeCascade = cv2.CascadeClassifier(cascPatheyes)

mask_image = cv2.imread('mickey.png')

In [5]:
video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h),(0,255,0), 2)
        
        faceROI = frame[y:y+h,x:x+w]
        eyes = eyeCascade.detectMultiScale(faceROI)
        
        for (x2, y2, w2, h2) in eyes:
            eye_center = (x + x2 + w2 // 2, y + y2 + h2 // 2)
            radius = int(round((w2 + h2) * 0.25))
            frame = cv2.circle(frame, eye_center, radius, (255, 0, 0), 4)

        mask_image_res = cv2.resize(mask_image, (w, h))
        mask_image_gray = cv2.cvtColor(mask_image_res, cv2.COLOR_BGR2GRAY)
        _, face_mask = cv2.threshold(mask_image_gray, 25, 255, cv2.THRESH_BINARY_INV)
        face_mask_no_face = cv2.bitwise_and(faceROI, faceROI, mask=face_mask)
        final_face_mask = cv2.add(face_mask_no_face, mask_image_res)
        
        frame[y:y+h,x:x+w] = final_face_mask
        
        # Display the resulting frame
        cv2.imshow("Filter Window", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [6]:
video_capture.release()
cv2.destroyAllWindows()